In [3]:
import utility as ut
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore

In [4]:
def load_matrices(filename="matrices.npz"):
    """
    Loads P and Phi matrices from a .npz file.

    Parameters:
    - filename: The filename to load (default: "matrices.npz").

    Returns:
    - P: Loaded storage function matrix.
    - Phi: Loaded supply rate function matrix.
    """
    data = np.load(filename)
    P = data["P"]
    Phi = data["Phi"]
    print(f"Loaded matrices from {filename}")
    return P, Phi

In [5]:
P, Phi = load_matrices(filename="matrices.npz")

Loaded matrices from matrices.npz


In [14]:
# Check if matrix P is positive definite
def check_positive_definite(P):
    """
    Checks if a matrix is positive definite.

    Parameters:
    - P: The matrix to check.

    Returns:
    - True if the matrix is positive definite, False otherwise.
    """
    return np.all(np.linalg.eigvals(P) > 0)

print(f"Matrix P is positive definite: {check_positive_definite(P)}")
print(f"Matrix Phi is positive definite: {check_positive_definite(Phi)}")

Matrix P is positive definite: True
Matrix Phi is positive definite: False


In [7]:
print(P.shape)
print(Phi.shape)

(10, 10)
(10, 10)


In [8]:
# Load data
DIRECTORY = "data/"
datasets = ut.load_excel_data(DIRECTORY)

In [9]:
DURATION = 15  # duration in seconds
DT = 0.02  # time increment in seconds
# Create a time array
time_array = np.arange(0, DURATION + DT, DT)
NUM_STEPS = len(time_array)

In [10]:
interpolated_datasets = []
for i in range(len(datasets)):
    new_trolley_position = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["trolley_position"]
    )
    new_cable_length = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["cable_length"]
    )
    new_sway_angle = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["sway_angle"]
    )
    new_trolley_motor_pwm = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["pwm_trolley_motor"]
    )
    new_hoist_motor_pwm = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["pwm_hoist_motor"]
    )
    interpolated_df = {
        "trolley_position": new_trolley_position,
        "cable_length": new_cable_length,
        "sway_angle": new_sway_angle,
        "trolley_motor_pwm": new_trolley_motor_pwm,
        "hoist_motor_pwm": new_hoist_motor_pwm,
    }

    interpolated_datasets.append(interpolated_df)

print(len(interpolated_datasets))


39


In [11]:
def dissipativity_inequality(P, Phi, xi0, xi1, Z):
    """
    Computes the dissipativity inequality for fault detection.

    Parameters:
    - P: Storage function matrix.
    - Phi: Supply rate function matrix.
    - xi0: Column vector for xi(k).
    - xi1: Column vector for xi(k+1).
    - Z: Column vector for Z(k).

    Returns:
    - True if the dissipativity inequality is satisfied, False otherwise.
    """
    # Compute the left-hand side of the inequality
    lhs = xi1.T @ P @ xi1 - xi0.T @ P @ xi0

    # Compute the right-hand side of the inequality
    rhs = Z.T @ Z

    # Check if the inequality is satisfied
    return lhs - rhs

In [12]:
L = 5  # Number of past steps to include
N = 5  # Number of future steps to predict
T = NUM_STEPS # Number of time steps
num_inputs = 2  # Number of inputs
num_outputs = 3  # Number of outputs

TF = 10 # window of samples observed for detecting faults

In [13]:
xi_list = []
zeta_list = []
Y_array = []
J_array = []
Y_square_array = []

for data in interpolated_datasets:
    for k in range (L, T - N):
        k -= 1
        past_outputs = np.column_stack([data["trolley_position"][k-L:k], data["cable_length"][k-L:k], data["sway_angle"][k-L:k]])
        past_inputs = np.column_stack([data["trolley_motor_pwm"][k-L:k], data["hoist_motor_pwm"][k-L:k]])

        xi = np.column_stack([past_outputs, past_inputs]).reshape(-1, 1)
        xi_list.append(xi)

        future_outputs = np.column_stack([data["trolley_position"][k:k+N], data["cable_length"][k:k+N], data["sway_angle"][k:k+N]])
        future_inputs = np.column_stack([data["trolley_motor_pwm"][k:k+N], data["hoist_motor_pwm"][k:k+N]])

        zeta = np.column_stack([future_outputs, future_inputs]).reshape(-1, 1)
        zeta_list.append(zeta)

        if k > L:
            Y = dissipativity_inequality(P, Phi, xi_list[k-L], xi_list[k-L+1], zeta_list[k-L])
            Y_array.append(Y)
            Y_square_array.append(Y_array[k-L-1]**2)

            if len(Y_square_array) > TF:
                J_array.append(np.sqrt(np.sum(Y_square_array[k-TF-L:k-L])))


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 25)

In [ ]:
# Calculate the mean of the fault detection metric
J_mean = np.mean(J_array)
print(f"Mean of the fault detection metric: {J_mean}")

# Calculate the standard deviation of the fault detection metric
J_std = np.std(J_array)
print(f"Standard deviation of the fault detection metric: {J_std}")

# Determine the threshold for fault detection
threshold = J_mean + 3 * J_std
print(f"Threshold for fault detection: {threshold}")

In [ ]:
plt.plot(np.linspace(0, 10, len(Y_array)), np.array(Y_array).flatten())

In [ ]:
print(Y_array)

In [ ]:
plt.plot(np.linspace(0, 10, len(Y_square_array)), np.array(Y_square_array).flatten())

In [ ]:
plt.plot(np.linspace(0, 10, len(J_array)), np.array(J_array).flatten())
plt.plot(np.linspace(0, 10, len(J_array)), np.linspace(threshold, threshold, len(J_array)))
plt.plot(np.linspace(0, 10, len(J_array)), np.linspace(J_mean, J_mean, len(J_array)))